In [ ]:
# Tensorflow Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory

# Plotting and Model Evaulation Libraries
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# Utility Libraries
import numpy as np
import os

# Setting seed for reproducibility
keras.utils.set_random_seed(42) 

**DATA LOADING**

In [ ]:
# We load the data from the GTZAN Dataset

training_images_filepath = "/GTZAN_Dataset/Data/images_original"
category_labels = os.listdir(training_images_filepath)

xdim = 180 
ydim = 180

# we also imported the spectograms provided in the GTZAN dataset

spectograms = image_dataset_from_directory(
    training_images_filepath,
    image_size = (xdim, ydim),
    batch_size = 111,
    label_mode='int'
)


## Use num_batches - 2 batches for training, 1 batch for validation, 1 batch for testing
num_batches = tf.data.experimental.cardinality(spectograms).numpy()
train = spectograms.take(num_batches - 2).cache()
remaining = spectograms.skip(num_batches - 2)
validation = remaining.take(1).cache()
test = remaining.skip(1).cache()

In [ ]:
# this display the spectograms

for images, labels in validation:
    plt.figure(figsize=(15, 500))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Label: {category_labels[labels[i].numpy()]}")
        plt.axis("off")
    plt.show()

**MODEL FINE-TUNING**

In [ ]:
# Here we have played around with different networks
# VGG16, ResNet50 and MobileNet
# The results for each of these models are documented in the report

conv_base = keras.applications.vgg16.VGG16(
    weights = "imagenet",
    include_top = False,
    input_shape = (xdim, ydim, 3))
conv_base.summary()

# conv_base = keras.applications.ResNet50(
#    weights = "imagenet",
#    include_top = False,
#    input_shape = (xdim, ydim, 3))
# conv_base.summary()

# conv_base = keras.applications.MobileNet(
#    weights = "imagenet",
#    include_top = False,
#    input_shape = (xdim, ydim, 3))
# conv_base.summary()

In [ ]:
# We freeze the layers and add in our own head layers to train first

conv_base.trainable = False 

# Define the new head
inputs = keras.Input(shape=(xdim, ydim, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(inputs)
x = layers.Flatten()(x)
# here we have played around with a few activation functions
# ReLU, elu and Sigmoid
x = layers.Dense(256, activation = "relu")(x)
outputs = layers.Dense(len(category_labels), activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

history = model.fit(
    train,
    epochs = 50,
    validation_data = validation,
    verbose = 0)

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Training Set", "Validation Set"]);

In [ ]:
# After our head layers are done training, we unfreeze the rest of the layers
# And then train them all together again

conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

history = model.fit(
    train,
    epochs = 10,
    validation_data = validation,
    verbose = 0)

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Training Set", "Validation Set"]);

**MODEL EVALUATION**

In [ ]:
# The model evaluation metrics we worked with
# are accuracy metrics, classification report and confusion matrix

predictions_prob = model.predict(test)
predictions = np.argmax(predictions_prob, axis = 1)

ground_truth = [label for _, label in test.unbatch()]
ground_truth = tf.stack(ground_truth, axis = 0).numpy()  

accuracy = accuracy_score(ground_truth, predictions)
print("Accuracy of the model:", accuracy)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
conf_matrix = confusion_matrix(ground_truth, predictions)
ConfusionMatrixDisplay(conf_matrix, display_labels = category_labels).plot(ax = ax)

report = classification_report(ground_truth, predictions)
print('\nClassification Report:\n', report)